In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
harry1=pd.read_table('Harry Potter 1.csv',sep=';',encoding='iso-8859-1')
harry1[:100]

,Character,Sentence
0,Dumbledore,"I should've known that you would be here, Prof..."
1,McGonagall,"Good evening, Professor Dumbledore."
2,McGonagall,"Are the rumors true, Albus?"
3,Dumbledore,"I'm afraid so, professor."
4,Dumbledore,The good and the bad.
...,...,...
95,Vernon,Proudest moment of my life.
96,Harry,Will I have to wear that too?
97,Petunia,What? You? Go to Smeltings?
98,Petunia,Don't be so stupid!
